In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from tkinter import Tk
from tkinter.filedialog import askdirectory


# Open folder picker
Tk().withdraw()
folder_path = askdirectory(title="Select Folder with CSV Files")

# List CSV files
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
#list of peak integral
peak_areas = [] 
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)

    try:
        # Skip the first row and use the second row as header
        df = pd.read_csv(file_path, skiprows=1, encoding='utf-8')

        time_unit = df.columns[0].strip('() ')
        voltage_unit = df.columns[1].strip('() ').lower()
        voltage_raw = df.iloc[:, 1] 
        #print(voltage_unit)


        # Rename columns
        df.columns = ['Time_us', 'Voltage_V']

        # Convert to numeric
        time_us = pd.to_numeric(df['Time_us'], errors='coerce')
        voltage = pd.to_numeric(df['Voltage_V'], errors='coerce')
        
        # Convert voltage to volts
        if 'mv' == voltage_unit:
            voltage = voltage_raw*1e-3
        elif 'v' == voltage_unit:
            voltage = voltage_raw
        else:
            raise ValueError(f"Unknown voltage unit: {voltage_unit}")
            
        # Convert time from microseconds to seconds
        time_s = time_us * 1e-6
        mode_voltage = stats.mode(voltage, keepdims=True)[0][0]

        voltage_adjusted = voltage - mode_voltage

        # Compute area
        area = np.trapz(voltage_adjusted, time_s)
        peak_areas.append(area)
        #print('mode_voltage: ' + str(mode_voltage) + ' V   Peak area: ' + str(area) + ' V·s')

    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        
#average peak integral value of selected data
print(len(peak_areas))
print(str(np.mean(peak_areas))+ ' V·s')

1000
1.883330374885383e-05 V·s
